In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np

In [2]:
from collections import Counter
from imblearn.datasets import fetch_datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

In [3]:
df = pickle.load(open('dataframe.pickle', 'rb'))
print(df.dtypes)
df.head()

issuercountrycode                object
txvariantcode                    object
bin                             float64
amount                          float64
currencycode                     object
shoppercountrycode               object
shopperinteraction               object
cardverificationcodesupplied     object
cvcresponsecode                   int64
creationdate                     object
accountcode                      object
mail_id                           int64
ip_id                             int64
card_id                           int64
label                             int64
creationdate_timestamp          float64
dtype: object


,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationdate_timestamp
196377,GB,visagold,494120.0,5227.0,GBP,GB,ContAuth,False,0,2015-10-1,UKAccount,273151,250291,216279,0,1.443726e+09
195925,GB,visadebit,492181.0,6895.0,GBP,GB,Ecommerce,NaN,1,2015-10-1,UKAccount,194157,13886,19361,0,1.443708e+09
195926,GB,visadebit,465858.0,8695.0,GBP,GB,Ecommerce,NaN,1,2015-10-1,UKAccount,230463,52387,106890,0,1.443708e+09
195927,GB,visadebit,476367.0,21700.0,GBP,GB,Ecommerce,True,1,2015-10-1,UKAccount,145581,43039,177365,0,1.443708e+09
195928,HK,visaplatinum,455728.0,9995.0,GBP,HK,Ecommerce,True,1,2015-10-1,UKAccount,90133,25944,175345,0,1.443708e+09


In [4]:
# TODO: to verify, does this dataframe takes into account Refused transactions?
# How many entries in total ?
df.shape[0]

236698

In [15]:
# How many of those are fraudolent and how many are safe ?
s = df.label.value_counts()
print s
print s[1]/float(s[0])

0    236353
1       345
Name: label, dtype: int64
0.0014596810702635465


In [27]:
# TODO clean and filter only the interesting features, this is just an example:
df_interesting = df[['label','ip_id','issuercountrycode']]
# Convert categorical variable into dummy/indicator variables for learning
df_encoded = pd.get_dummies(df_interesting)
df_encoded.head()

,label,ip_id,issuercountrycode_AE,issuercountrycode_AI,issuercountrycode_AL,issuercountrycode_AM,issuercountrycode_AO,issuercountrycode_AR,issuercountrycode_AT,issuercountrycode_AU,...,issuercountrycode_TZ,issuercountrycode_UA,issuercountrycode_US,issuercountrycode_UY,issuercountrycode_VG,issuercountrycode_VN,issuercountrycode_ZA,issuercountrycode_ZM,issuercountrycode_ZW,issuercountrycode_ZZ
196377,0,250291,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195925,0,13886,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195926,0,52387,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195927,0,43039,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195928,0,25944,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Creating training and test sets
training_features, test_features, \
training_target, test_target, = train_test_split(df_encoded.drop(['label'], axis=1),
                                               df_encoded['label'],
                                               test_size = .1,
                                               random_state=12)


In [40]:
# SMOTE analysis (note, this is done after sampling)
sm = SMOTE(random_state=12)
x_res, y_res = sm.fit_sample(training_features, training_target)
print training_target.value_counts()
print np.bincount(y_res)


0    212725
1       303
Name: label, dtype: int64
[212725 212725]
